In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.redfinclean
df = pd.DataFrame(list(db.redfin.find().limit(10000)))

In [3]:
sel_cols = ['period_begin',  
'period_end', 
'region_name' ,
'region_type',
'total_homes_sold',
'median_sale_price',
'median_sale_ppsf'
]

In [4]:
df_reduced = df[sel_cols]

In [5]:
# df_reduced["period_begin"].to_datetime()
# 
begin_dates = pd.to_datetime(df_reduced["period_begin"], infer_datetime_format=True)
df_reduced["year"] = begin_dates.dt.strftime("%Y")

df_reduced["year_month"] = begin_dates.dt.strftime("%Y-%m")

states = []
for region in df_reduced["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
df_reduced["state"] = states

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [6]:
df_nometro = df_reduced.loc[df_reduced["region_type"] != "metro"]

In [7]:
df_nometro.head()

,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,median_sale_ppsf,year,year_month,state
0,2019-09-09,2019-09-15,"Haywood County, NC",county,34.0,240000.0,143.20260577745609,2019,2019-09,NC
1,2018-11-05,2018-11-11,"Eaton County, MI",county,19.0,187500.0,121.765601217656,2018,2018-11,MI
2,2020-07-20,2020-07-26,"Greenville County, SC",county,207.0,255000.0,121.04283054003724,2020,2020-07,SC
3,2019-02-04,2019-02-10,"Dane County, WI",county,65.0,274000.0,147.58389261744966,2019,2019-02,WI
4,2017-05-01,2017-05-07,"Aroostook County, ME",county,8.0,58700.0,45.265126771662885,2017,2017-05,ME


In [8]:
df_nometro["region_type"].value_counts()


county    7921
Name: region_type, dtype: int64

In [9]:

df_nometro.fillna('', inplace=True)
df_nometro.head()


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,median_sale_ppsf,year,year_month,state
0,2019-09-09,2019-09-15,"Haywood County, NC",county,34.0,240000.0,143.20260577745609,2019,2019-09,NC
1,2018-11-05,2018-11-11,"Eaton County, MI",county,19.0,187500.0,121.765601217656,2018,2018-11,MI
2,2020-07-20,2020-07-26,"Greenville County, SC",county,207.0,255000.0,121.04283054003724,2020,2020-07,SC
3,2019-02-04,2019-02-10,"Dane County, WI",county,65.0,274000.0,147.58389261744966,2019,2019-02,WI
4,2017-05-01,2017-05-07,"Aroostook County, ME",county,8.0,58700.0,45.265126771662885,2017,2017-05,ME


In [10]:
final_cols = ['year',
              'state',
'total_homes_sold',
'median_sale_price',
]

In [11]:
df_redfinclean = df_nometro[final_cols]

In [12]:
data_dict = df_redfinclean.to_dict("records")

In [13]:
collection.insert_many(data_dict)